In [1]:
## Installing required libraries
!pip install langchain langchain-core langchain-community pydantic duckduckgo-search -q


[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [34]:
## Importing required libraries
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain_perplexity import ChatPerplexity
from langchain.tools import tool
import requests

In [35]:
## Calling in buit tool 

search_tool = DuckDuckGoSearchRun()

result = search_tool.invoke("Top 10 news in India today?")

print(result)

c:\Users\DELL\Desktop\LangChain\venv\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


Mar 8, 2025 · Myntra is home to some of the most on-trend top styles of the season. You can browse, filter, and shop by your favourite global and domestic brands at the price you are … Tops - Shop Tops at India's Best Online Shopping Store. Check Price and Buy Online. Free Shipping Cash on Delivery Best Offers Top news stories from world, politics, business, sports, entertainment, lifestyle online. Find 1582 different ways to say TOP, along with antonyms, related words, and example sentences at Thesaurus.com. Top Charts ranks the top 100 songs of the week based on Apple Music, Youtube and Spotify streaming, iTunes sales, and also radio airplay on myTuner app.


In [55]:
## Creating a second tool that fetches temperature 

@tool
def get_weather_data(city: str) -> str:
    '''
    This function fetches the  current weather data for a given city
    '''

    url = f'http://api.weatherstack.com/current?access_key=9085b172d13e26b6515a4c1e67de06b9&query={city}'

    request = requests.get(url)

    return request.json()


In [56]:
## Initializing the LLM 

llm = HuggingFaceEndpoint(
    repo_id= 'moonshotai/Kimi-K2-Instruct'
    )

model = ChatHuggingFace(llm = llm)


# model = ChatPerplexity(model="sonar") 

In [57]:
## Testing LLM if its working fine
model.invoke("Hi")

AIMessage(content='Hi there! How can I help you today?', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 8, 'total_tokens': 19}, 'model_name': 'moonshotai/Kimi-K2-Instruct', 'system_fingerprint': '', 'finish_reason': 'stop', 'logprobs': None}, id='run--53ef4513-66a6-45c0-a67b-0bcad46ddb1d-0', usage_metadata={'input_tokens': 8, 'output_tokens': 11, 'total_tokens': 19})

In [58]:
## Now we have a tool (duckduckgo) and a llm (huggingface), we are now set to build out first agent

In [59]:
## Importing libraries needed for AI Agents
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub

In [60]:
## Step 2: Pull the RecAct (Reasoning and Action) prompt from langchain
prompt = hub.pull("hwchase17/react")

c:\Users\DELL\Desktop\LangChain\venv\Lib\site-packages\langsmith\client.py:272: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [61]:
## Step 3: Create a ReAct agent manually with the pulled prompt
agent = create_react_agent(

    llm= model,
    tools= [search_tool, get_weather_data],
    prompt= prompt
)

In [62]:
## Step 4: Wrap it with AgentExecutor
agent_executor = AgentExecutor(
    agent= agent,
    tools= [search_tool, get_weather_data],
    verbose= True
)

In [63]:
## Step 5: Invoke
result= agent_executor.invoke({"input" : "Find the capital of Utrakhand, then find its current weather condition?"})
print(result)



> Entering new AgentExecutor chain...
I need to find the capital of Uttarakhand first, then get its current weather.

Action: duckduckgo_search
Action Input: capital of Uttarakhand

c:\Users\DELL\Desktop\LangChain\venv\Lib\site-packages\langchain_community\utilities\duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


The state is divided into two divisions, Garhwal and Kumaon, with a total of 13 districts. The capital of Uttarakhand is Dehradun, the largest city of the state, which is a railhead. The High Court of … This webpage provides a solution to a physics problem involving charges at the vertices of an equilateral triangle. Learn the concepts of Class 8 General Knowledge Overview of India with Videos and Stories. Capital of Indian States and Union Territories.,History of Capital of Indian States and Union … Chhattisgarh is a heavily forested state in central India known for its temples and waterfalls. Near the capital Raipur, the town of Sirpur on the Mahanadi River is home to the red-brick Lakshmana … The capital of Uttarakhand is Dehradun. Dehradun is also the largest city here. It has a total area of 53,483km/20,650mi and a population of 10,116,752. In this state, Hindi and Sanskrit (!) are the …Now I know the capital of Uttarakhand is Dehradun. I need to get the current weather data for D

In [64]:
print(result['output'])

The capital of Uttarakhand is **Dehradun**. As of **August 7, 2025, 12:52 AM local time**, the current weather in Dehradun is **21°C** with **partly cloudy** skies, 91% humidity, and a northeast wind at 5 km/h.
